In [1]:
from datasets import load_dataset
ds = load_dataset("knkarthick/samsum")

In [2]:
traindataset=ds["train"]

In [3]:
traindataset

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [4]:
dialogue = traindataset['dialogue']

In [5]:
dialogue

["Amanda: I baked  cookies. Do you want some?\nJerry: Sure!\nAmanda: I'll bring you tomorrow :-)",
 'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great',
 "Tim: Hi, what's up?\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\nTim: What did you plan on doing?\nKim: Oh you know, uni stuff and unfucking my room\nKim: Maybe tomorrow I'll move my ass and do everything\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\nTim: It really helps\nKim: thanks, maybe I'll do that\nTim: I also like using post-its in kaban style",
 "Edward: Rachel, I think I'm in ove with Bella..\nrachel: Dont say anything else..\nEdward: What do you mean??\nrachel: Open your fu**ing door.. I'm outside",
 "Sam: hey  overheard rick say something\nSam: i don't know what to do :-/\nNaomi: what did 

In [6]:
import re

def reshape(dataset):
    reshape_dataset = [0] * len(dataset)
    for i in range(len(dataset)):
        reshape_dataset[i]=dataset[i]['dialogue']
    
    for i, item in enumerate(reshape_dataset):
        if not isinstance(item, str) :
            print(f"Index {i}: {type(item)} - {item}")
    reshape_dataset = [item for item in reshape_dataset if isinstance(item, str)]
    reshape_dataset = [re.sub(r'\s', ' ', item) for item in reshape_dataset]
    reshape_dataset = [re.sub(r"[^a-zA-Z0-9 .:)!?'-]", '', item) for item in reshape_dataset]

    return reshape_dataset

In [7]:
dataset=reshape(traindataset)

Index 6054: <class 'NoneType'> - None


In [8]:
traindataset['dialogue']

["Amanda: I baked  cookies. Do you want some?\nJerry: Sure!\nAmanda: I'll bring you tomorrow :-)",
 'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great',
 "Tim: Hi, what's up?\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\nTim: What did you plan on doing?\nKim: Oh you know, uni stuff and unfucking my room\nKim: Maybe tomorrow I'll move my ass and do everything\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\nTim: It really helps\nKim: thanks, maybe I'll do that\nTim: I also like using post-its in kaban style",
 "Edward: Rachel, I think I'm in ove with Bella..\nrachel: Dont say anything else..\nEdward: What do you mean??\nrachel: Open your fu**ing door.. I'm outside",
 "Sam: hey  overheard rick say something\nSam: i don't know what to do :-/\nNaomi: what did 

In [110]:
dataset

["Amanda: I baked  cookies. Do you want some? Jerry: Sure! Amanda: I'll bring you tomorrow :-)",
 'Olivia: Who are you voting for in this election?  Oliver: Liberals as always. Olivia: Me too!! Oliver: Great',
 "Tim: Hi what's up? Kim: Bad mood tbh I was going to do lots of stuff but ended up procrastinating Tim: What did you plan on doing? Kim: Oh you know uni stuff and unfucking my room Kim: Maybe tomorrow I'll move my ass and do everything Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores Tim: It really helps Kim: thanks maybe I'll do that Tim: I also like using post-its in kaban style",
 "Edward: Rachel I think I'm in ove with Bella.. rachel: Dont say anything else.. Edward: What do you mean?? rachel: Open your fuing door.. I'm outside",
 "Sam: hey  overheard rick say something Sam: i don't know what to do :- Naomi: what did he say?? Sam: he was talkin

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import re
from tqdm import tqdm
import torch
from torch.nn import functional as F
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn



def reshape(dataset):
    reshape_dataset = [0] * len(dataset)
    for i in range(len(dataset)):
        if isinstance(dataset[i]["dialogue"], str) and isinstance(dataset[i]["summary"], str):
            reshape_dataset[i]="C: "+dataset[i]["dialogue"]+" S: "+dataset[i]["summary"]
    reshape_dataset = [item for item in reshape_dataset if isinstance(item, str)]
    reshape_dataset = [re.sub(r'\s+', ' ', item) for item in reshape_dataset]
    reshape_dataset = [re.sub(r'[^a-zA-Z0-9 .:?]', '', item) for item in reshape_dataset]
 
    return reshape_dataset[:data_size]

def max_length(dataset):
    max_eval=0
    for i in dataset:
        max_eval = len(i) if len(i) > max_eval else max_eval
    print(max_eval)
    return

def batch(input, size):
    batch_train=[]
    for i in range(size):
        batch_input=[input[4*i+0], input[4*i+1], input[4*i+2], input[4*i+3]]
        batch_train.append(batch_input)

    return batch_train

def make_data(data):
    dataset=reshape(data)
    data = []
    for text in tqdm(dataset, desc="Tokenizing dataset"):
        cq_len=len(tokenizer(text[:text.find(" S:")])['input_ids'])
        tokenized = tokenizer(text, padding="max_length", max_length=512, truncation=True, return_tensors="pt")
        input_ids = tokenized['input_ids'].squeeze().tolist()
        attention_mask = tokenized['attention_mask'].squeeze().tolist()
        labels = input_ids[1:] + [tokenizer.pad_token_id]
        for i in range(min(cq_len+1, 512)):
            labels[i]=128001
        data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})
    
    return data

def make_tensor(data, type, size):
    tmp = [item[type] for item in data]
    tmp = batch(tmp, size)
    tensor=torch.tensor(tmp, dtype=torch.long)
    return tensor

ds = load_dataset("knkarthick/samsum")
device='cuda'
model = AutoModelForCausalLM.from_pretrained("../model/distill_model")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# for name, param in model.named_parameters():
#     if 'lora_A' in name or 'lora_B' in name:
#         param.requires_grad = True

data_size = 1000
data_size_v = 100
size = int(data_size/4)
size_v = int(data_size_v/4)
train_dataset=ds["train"].shuffle(seed=42).select(range(10000))
validation_dataset=ds["validation"].shuffle(seed=42)

data = make_data(train_dataset)
data_v = make_data(validation_dataset)

input_ids_tensor = make_tensor(data, "input_ids", size)
labels_tensor = make_tensor(data, "labels", size)
attention_mask_tensor = make_tensor(data, "attention_mask", size)
input_ids_tensor_v = make_tensor(data_v, "input_ids", size_v)
labels_tensor_v = make_tensor(data_v, "labels", size_v)
attention_mask_tensor_v = make_tensor(data_v, "attention_mask", size_v)


vocab_size = model.config.vocab_size
criterion = torch.nn.CrossEntropyLoss(ignore_index=128001)

criterion.to(device)
input_ids_tensor=input_ids_tensor.to(device)
labels_tensor=labels_tensor.to(device)
attention_mask_tensor=attention_mask_tensor.to(device)
input_ids_tensor_v=input_ids_tensor_v.to(device)
labels_tensor_v=labels_tensor_v.to(device)
attention_mask_tensor_v=attention_mask_tensor_v.to(device)

model.to(device)

epochs = 3
lr=1e-7

model.train()

# for name, param in model.named_parameters():
#     print(f"{name}: requires_grad={param.requires_grad}")


for j in range(epochs):
    optimizer = AdamW(model.parameters(), lr=lr)
    for i in tqdm(range(size)):
        
        input_ids=input_ids_tensor[i]
        labels=labels_tensor[i]
        attention_mask=attention_mask_tensor[i]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits=outputs.logits
        loss = criterion(logits.view(-1, vocab_size), labels.view(-1))
        loss.backward()
        optimizer.step()
        
    print("done: ", j+1, "/", epochs)
    lr/=10

model.save_pretrained("./model/train_test")







Tokenizing dataset: 100%|██████████| 100/100 [00:00<00:00, 908.43it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [4]:
arr =[data.item() for items in input_ids_tensor for item in items for data in item if data != 128001]

In [26]:
labels = [[item for item in raw if item != 128001] for row in labels_tensor.tolist() for raw in row]

In [23]:
print(tokenizer.decode(328))

 S


In [27]:
labels

[[25, 48749, 44204, 3821, 94326, 904, 810, 439, 1202, 54766, 13],
 [25,
  1115,
  9178,
  55676,
  374,
  1633,
  16053,
  1606,
  1364,
  6575,
  2653,
  520,
  990,
  323,
  38243,
  374,
  6574,
  11517,
  13,
  22021,
  374,
  62724,
  304,
  279,
  13863,
  323,
  19354,
  813,
  10747,
  19826,
  6307,
  13,
  9479,
  2046,
  7505,
  38243,
  11517,
  323,
  55676,
  690,
  4034,
  1461,
  304,
  813,
  502,
  10980,
  2908,
  13],
 [25,
  71566,
  2663,
  85037,
  719,
  1364,
  48707,
  4320,
  1606,
  374,
  304,
  46232,
  682,
  1938,
  13,
  85037,
  690,
  1650,
  71566,
  1203,
  2391,
  279,
  1464,
  13,
  71566,
  649,
  3350,
  311,
  85037,
  1606,
  1364,
  649,
  1373,
  323,
  3350,
  2391,
  279,
  31678,
  13,
  85037,
  16869,
  3137,
  477,
  9020,
  2391,
  279,
  31678,
  13,
  220],
 [25,
  91979,
  6944,
  311,
  733,
  12185,
  389,
  7418,
  449,
  24101,
  13,
  24101,
  16869,
  5249,
  1077,
  1606,
  1364,
  706,
  2736,
  6787,
  369,
  279,
  4459,

In [20]:
print(len(labels))

100


In [28]:
tokenizer.decode(13)

'.'